# AnimateAI - Image to Video Animation (T4 GPU Version)

This notebook allows you to run the AnimateAI application directly in Google Colab with T4 GPU acceleration.

## Setup Instructions
1. Make sure to select **Runtime > Change runtime type > Hardware accelerator > GPU** in the Colab menu.
2. Run all cells in sequence to set up the environment and start the application.
3. Follow the link provided at the end to access the web interface.

## 1. Check GPU Availability

First, let's verify that we have access to a GPU (ideally T4).

In [ ]:
!nvidia-smi

## 2. Clone the AnimateAI Repository

Let's clone the repository from GitHub.

In [ ]:
!git clone https://github.com/YOUR_USERNAME/animate-ai.git
%cd animate-ai

## 3. Install Dependencies

Now we'll install all the required dependencies for both the web application and AI components.

In [ ]:
# Install Python dependencies for the AI backend
!pip install torch==2.0.1 torchvision==0.15.2 diffusers==0.23.1 transformers==4.33.2 accelerate==0.23.0 safetensors==0.3.3 imageio==2.31.5 flask==2.3.3 flask-cors==4.0.0 pillow==10.0.1 huggingface_hub==0.17.3 -q

# Install Node.js and npm
!apt-get update
!apt-get install -y nodejs npm

# Install frontend dependencies
%cd react_template
!npm install
%cd ..

## 4. Build the Frontend

Let's build the React application.

In [ ]:
%cd react_template
!npm run build
%cd ..

## 5. Login to Hugging Face (Optional)

Some models might require authentication with Hugging Face.

In [ ]:
from huggingface_hub import notebook_login

# Uncomment and run if you need to access gated models
# notebook_login()

## 6. Initialize the Stable Video Diffusion Model

Now, let's initialize the AI model that will convert images to videos.

In [ ]:
# Import the SVD integration module
import sys
sys.path.append('./colab')
from stable_video_diffusion_integration import StableVideoDiffusionService

# Initialize the SVD service - this will download the model (may take a few minutes)
svd_service = StableVideoDiffusionService()
svd_service.initialize()

## 7. Create a Simple Test

Let's test the model with a sample image to make sure everything is working correctly.

In [ ]:
from IPython.display import Image, display
import os
import urllib.request

# Create a test directory
!mkdir -p test_images

# Download a test image
test_image_url = "https://images.unsplash.com/photo-1581833971358-2c8b550f87b3?ixlib=rb-4.0.3&q=85&fm=jpg&crop=entropy&cs=srgb&w=1024"
test_image_path = "test_images/mountain.jpg"
urllib.request.urlretrieve(test_image_url, test_image_path)

# Display the test image
display(Image(filename=test_image_path))
print("Test image downloaded and displayed.")

In [ ]:
# Generate a test video
output_path = svd_service.generate_video(
    image_path=test_image_path,
    motion_strength=0.5,
    duration=2.0,
    fps=24
)

# Display the generated video in the notebook
svd_service.display_video_in_notebook(output_path)

## 8. Start the Backend API Server

Now we'll start the Flask API server that will handle requests from the frontend.

In [ ]:
# Install pyngrok to create a public URL for our API
!pip install pyngrok -q

from pyngrok import ngrok
import threading
from stable_video_diffusion_integration import run_api_server

# Start ngrok tunnel
public_url = ngrok.connect(5000)
print(f"API server will be available at: {public_url}")

# Start API server in a separate thread
api_thread = threading.Thread(target=run_api_server, daemon=True)
api_thread.start()

# Wait for the server to start
import time
time.sleep(3)
print("API server is now running!")

## 9. Serve the Frontend with API Integration

Finally, we'll serve the React frontend with proper API configuration.

In [ ]:
# Create a simple server to serve the frontend
import http.server
import socketserver
import threading
import os

# Update the API URL in the frontend build
api_url = public_url.public_url
frontend_config = f"""
window.API_CONFIG = {{
  "apiUrl": "{api_url}"
}};
"""

with open("react_template/dist/config.js", "w") as f:
    f.write(frontend_config)

# Set up a simple HTTP server for the frontend
class FrontendServer:
    def __init__(self, port=8000):
        self.port = port
        self.httpd = None
    
    def start(self):
        os.chdir("react_template/dist")
        handler = http.server.SimpleHTTPRequestHandler
        self.httpd = socketserver.TCPServer(("", self.port), handler)
        print(f"Serving frontend at port {self.port}")
        self.httpd.serve_forever()

# Start frontend server
server = FrontendServer(port=8000)
frontend_thread = threading.Thread(target=server.start, daemon=True)
frontend_thread.start()

# Create public URL for frontend
frontend_url = ngrok.connect(8000)
print(f"\n\n✅ AnimateAI is ready! Access the web application at: {frontend_url.public_url}\n")

## 10. Keep the Notebook Running

Execute the cell below to prevent Colab from timing out while you use the application.

In [ ]:
# Keep the notebook running
import IPython
from google.colab import output
import time

# Clear output every 60 seconds to keep the notebook running
while True:
    time.sleep(60)
    output.clear()
    IPython.display.display(IPython.display.HTML('''
        <div style="background-color: #4CAF50; color: white; padding: 10px; border-radius: 5px;">
            AnimateAI is still running. Keep this tab open to continue using the application.
        </div>
    '''))